# MATALA 3

### by : 
###      Shir Saadi

In [29]:
import pandas as pd
import requests

In [30]:
PATH = "C:\\Users\\shirs\\Desktop\\shir saadi\\שנה ג סמסטר ב\\כרייה וניתוח נתונים מתקדם בפייתון\\מטלות\\מטלה 3\\dests.txt"
data = pd.read_csv(PATH, names=['Target']) 
data.head()

,Target
0,Istanbul
1,Amsterdam
2,Valletta
3,Basel
4,Doha


#### פונקציה שמביאה את המרחק והזמן מתל אביב ליעד

In [31]:
def get_distance_and_duration(place):
    destination = str(place)
    API_KEY = "my api key"
    TEL_AVIV = str("Tel aviv")
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?destinations={destination}&origins={TEL_AVIV}&units=kilometer&key={API_KEY}"
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response = requests.get(url).json()
                distance = response["rows"][0]["elements"][0]["distance"]["value"] /1000
                duration = response["rows"][0]["elements"][0]["duration"]["value"] # אם שמים טקסט במקום ערך זה מחזיר סטריג של ימים ודקות ולא שעות ודקות לכן המרתי את הערך
                hours,seconds = divmod(duration, 3600) #פונקציה שממירה את השניות לשעות ומשאירה את השניות שנותרו כשארית
                minutes = seconds // 60
                duration_fix = f"{hours} hours {minutes} min"
                lst=[distance,duration_fix]
                return lst
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

data["Distance_km"] = data["Target"].apply(lambda x: get_distance_and_duration(x)[0])
data["Duration (hour+minutes)"] = data["Target"].apply(lambda x: get_distance_and_duration(x)[1])

#### פונקציה שמביאה את קו האורך והרוחב של מקום/כתובת

In [32]:
def get_longitude_and_latitude(place):
    API_KEY = "my api key"
    address =  str(place)
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={API_KEY}"
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response = requests.get(url).json()
                longitude = response["results"][0]["geometry"]["location"]["lng"]
                latitude = response["results"][0]["geometry"]["location"]["lat"]
                lst=[longitude,latitude]
                return lst
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

data["Longitude"] = data["Target"].apply(lambda x: get_longitude_and_latitude(x)[0])
data["Latitude"] = data["Target"].apply(lambda x: get_longitude_and_latitude(x)[1])

In [33]:
data

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,1815.227,21 hours 2 min,28.978359,41.008238
1,Amsterdam,4532.541,48 hours 3 min,4.904139,52.367573
2,Valletta,3792.883,50 hours 50 min,14.514100,35.899237
3,Basel,4092.805,44 hours 2 min,7.588576,47.559599
4,Doha,2164.477,22 hours 38 min,51.531040,25.285447


#### שלוש הערים הכי רחוקות מתל אביב

In [34]:
# מיון הנתונים לפי מרחק מתל אביב
sort_data = data.sort_values(by='Distance_km', ascending=False)
sort_data.head(3)

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
1,Amsterdam,4532.541,48 hours 3 min,4.904139,52.367573
3,Basel,4092.805,44 hours 2 min,7.588576,47.559599
2,Valletta,3792.883,50 hours 50 min,14.514100,35.899237
